# Tasks Templates

Hi there! In this article we wanted to share some examples of our supported tasks, so you can go from zero to hero as fast as possible. We are going to cover those task present in our [supported tasks list](https://docs.rubrix.ml/en/stable/getting_started/supported_tasks.html), so don't forget to stop by and take a look.

The tasks are divided into their different category, from text classification to token classification. We will update these article, as well as the supported task list, when a new task gets added to Rubrix.

## Text Classification

Text classification deals with predicting in which categories a text fits. As if you’re shown an image you could quickly tell if there’s a dog or a cat in it, we build NLP models to distinguish between a Jane Austen’s novel or a Charlotte Bronte’s poem. It’s all about feeding models with labeled examples and see how it start predicting over the very same labels.

### Text Categorization

This is a general example for the Text Classification family of tasks. Here, we will try to assign pre-defined categories to sentences and texts. Here, the possibilities are endless! Topic categorization, spam detection, and a vast etcétera.

For our example, we are using the [SequeezeBERT](https://huggingface.co/typeform/squeezebert-mnli) zero-shot classifier for predicting the topic of a given text, in three different labels: politics, sports and technology. We are also using [AG](https://huggingface.co/datasets/ag_news), a collections of news.

In [7]:
import rubrix as rb
from transformers import pipeline
from datasets import load_dataset

# Loading our dataset
dataset = load_dataset("ag_news", split="train[0:20]")

# Define our HuggingFace Pipeline
classifier = pipeline(
    "zero-shot-classification",
    model="typeform/squeezebert-mnli",
    framework="pt",
)

records = []

for record in dataset:

    # Making the prediction
    prediction = classifier(
        record["text"],
        candidate_labels=[
            "politics",
            "sports",
            "technology",
        ],
    )

    # Creating the prediction entity as a list of tuples (label, probability)
    prediction = list(zip(prediction["labels"], prediction["scores"]))

    # Appending to the record list
    records.append(
        rb.TextClassificationRecord(
            inputs=record["text"],
            prediction=prediction,
            prediction_agent="https://huggingface.co/typeform/squeezebert-mnli",
            metadata={"split": "train"},
        )
    )

# Logging into Rubrix
rb.log(
    records=records,
    name="text-categorization",
    tags={
        "task": "text-categorization",
        "phase": "data-analysis",
        "dataset": "ag_news",
    },
)

Using custom data configuration default
Reusing dataset ag_news (/Users/ignacio/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


BulkResponse(dataset='zeroshot-topic-classifier', processed=20, failed=0)

### Sentiment Analysis

In this kind of projects, we want our models to be able to detect the polarity of an input. Categories like *positive*, *negative* or *neutral* are often used. 

For this example, we are going to use an [Amazon review polarity dataset](https://huggingface.co/datasets/amazon_polarity), and a zero-shot [DistilBERT model](https://huggingface.co/typeform/distilbert-base-uncased-mnli). 

In [34]:
import rubrix as rb
from transformers import pipeline
from datasets import load_dataset

# Loading our dataset
dataset = load_dataset("amazon_polarity", split="train[0:20]")

# Define our HuggingFace Pipeline
classifier = pipeline(
    "zero-shot-classification",
    model="typeform/distilbert-base-uncased-mnli",
    framework="pt",
)

records = []

for record in dataset:

    # Making the prediction
    prediction = classifier(
        record["content"], candidate_labels=["positive", "negative"]
    )

    # Creating the prediction entity as a list of tuples (label, probability)
    prediction = list(zip(prediction["labels"], prediction["scores"]))

    # Appending to the record list
    records.append(
        rb.TextClassificationRecord(
            inputs=record["content"],
            prediction=prediction,
            prediction_agent="https://huggingface.co/typeform/distilbert-base-uncased-mnli",
            metadata={"split": "train"},
        )
    )

# Logging into Rubrix
rb.log(
    records=records,
    name="sentiment-analysis",
    tags={
        "task": "sentiment-analysis",
        "phase": "data-annotation",
        "dataset": "amazon-polarity",
    },
)

Reusing dataset amazon_polarity (/Users/ignacio/.cache/huggingface/datasets/amazon_polarity/amazon_polarity/3.0.0/ac31acedf6cda6bc2aa50d448f48bbad69a3dd8efc607d2ff1a9e65c2476b4c1)
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


BulkResponse(dataset='sentiment-analysis', processed=20, failed=0)